# 0. Import Library

In [1]:
import os
if not os.path.exists("./tfdet"):
    !git clone -q http://github.com/burf/tfdetection.git
    !mv ./tfdetection/tfdet ./tfdet
    !rm -rf ./tfdetection

In [2]:
#ignore warning
import warnings, os
warnings.filterwarnings(action = "ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tfdet

# 1. Init Dataset

In [3]:
#sample dataset
import numpy as np

#image_shape = [512, 512]
image_shape = [256, 256]
n_class = 21 #background + 20 label
batch_size = 16

def load(x_true, mask = True):
    max_pad_size = np.random.randint(10) + 1
    
    image = np.random.random([*image_shape, 3]).astype(np.float32)
    y_true = np.random.randint(n_class, size = [max_pad_size, 1]).astype(np.int32)
    bbox_true = np.array([tfdet.core.bbox.random_bbox(image_shape = image_shape) for _ in range(max_pad_size)], dtype = np.int32)
    result = [image, y_true, bbox_true]
    if mask:
        mask_true = np.random.randint(2, size = (max_pad_size, *image_shape, 1)).astype(np.uint8)
        result = [image, y_true, bbox_true, mask_true]
    return tuple(result)

bbox_dataset = tfdet.dataset.Dataset(np.arange(10), transform = [{"name":load, "mask":False}, #or load,
                                                                 {"name":"resize", "image_shape":image_shape},
                                                                 {"name":"pad", "image_shape":image_shape},
                                                                 {"name":"filter_annotation"},
                                                                 #{"name":"label_encode", "label":None},
                                                                 {"name":"normalize", "mean":[123.675, 116.28, 103.53], "std":[58.395, 57.12, 57.375]}])

mask_dataset = tfdet.dataset.Dataset(np.arange(10), transform = [{"name":load, "mask":True}, #or load,
                                                                 {"name":"resize", "image_shape":image_shape},
                                                                 {"name":"pad", "image_shape":image_shape},
                                                                 {"name":"filter_annotation"},
                                                                 #{"name":"label_encode", "label":None},
                                                                 {"name":"normalize", "mean":[123.675, 116.28, 103.53], "std":[58.395, 57.12, 57.375]},
                                                                 {"name":"mask_downscale", "scale":8}])

bbox_pipe = tfdet.dataset.PipeLoader(bbox_dataset)
bbox_pipe = tfdet.dataset.pipeline.args2dict(bbox_pipe, batch_size = batch_size, prefetch = True)

mask_pipe = tfdet.dataset.PipeLoader(mask_dataset)
mask_pipe = tfdet.dataset.pipeline.args2dict(mask_pipe, batch_size = batch_size, prefetch = True)

# 2. Build 1-Stage Object Detector

2-1. RetinaNet

In [4]:
#train model
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, frozen_stages = 1, weights = "imagenet_v2")

out = tfdet.model.detector.retinanet(feature, image_shape = tf.shape(x)[1:3], n_class = n_class, scale = [32, 64, 128, 256, 512], ratio = [0.5, 1, 2], octave = 3)

model = tfdet.model.train.retina.train_model(x, *out,
                                             proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)
model.compile()
model.evaluate(bbox_pipe)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

1/1 [==============================] - 4s 4s/step - loss: 5688.9619 - loss_class: 5678.9521 - loss_bbox: 10.0098


[(10, 100, 21), (10, 100, 4)]

In [5]:
#predict model
nms_out = tfdet.model.postprocess.retina.FilterDetection(proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)(out)
model = tf.keras.Model(x, nms_out)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

[(10, 100, 21), (10, 100, 4)]

2-2. EfficientDet

In [6]:
#train model
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
out = tfdet.model.detector.effdet_d4(x, n_class = n_class, scale = [32, 64, 128, 256, 512], ratio = [0.5, 1, 2], octave = 3, weights = "imagenet")

model = tfdet.model.train.effdet.train_model(x, *out,
                                             proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)
model.compile()
model.evaluate(bbox_pipe)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

1/1 [==============================] - 9s 9s/step - loss: 2017.4082 - loss_class: 2014.9301 - loss_bbox: 2.4767


[(10, 100, 21), (10, 100, 4)]

In [7]:
#predict model
nms_out = tfdet.model.postprocess.effdet.FilterDetection(proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)(out)
model = tf.keras.Model(x, nms_out)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

[(10, 100, 21), (10, 100, 4)]

2-3. EfficientDet-Lite

In [8]:
#train model
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
out = tfdet.model.detector.effdet_lite_d4(x, n_class = n_class, scale = [32, 64, 128, 256, 512], ratio = [0.5, 1, 2], octave = 3, weights = "imagenet")

model = tfdet.model.train.effdet.train_model(x, *out,
                                             proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)
model.compile()
model.evaluate(bbox_pipe)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

1/1 [==============================] - 6s 6s/step - loss: 4320.0464 - loss_class: 4317.6992 - loss_bbox: 2.3471


[(10, 100, 21), (10, 100, 4)]

In [9]:
#predict model
nms_out = tfdet.model.postprocess.effdet.FilterDetection(proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)(out)
model = tf.keras.Model(x, nms_out)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

[(10, 100, 21), (10, 100, 4)]

2-4. Yolo

In [10]:
#train model
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
out = tfdet.model.detector.yolo_v4(x, n_class = n_class, size = [[ 10, 13], [ 16,  30], [ 33,  23], 
                                                                 [ 30, 61], [ 62,  45], [ 59, 119], 
                                                                 [116, 90], [156, 198], [373, 326]], weights = "darknet")

model = tfdet.model.train.yolo.train_model(x, *out,
                                           proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)
model.compile()
model.evaluate(bbox_pipe)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

1/1 [==============================] - 3s 3s/step - loss: 302.2051 - loss_score: 291.7308 - loss_class: 7.7077 - loss_bbox: 2.7667


[(10, 100, 21), (10, 100, 4)]

In [11]:
#predict model
nms_out = tfdet.model.postprocess.yolo.FilterDetection(proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)(out)
model = tf.keras.Model(x, nms_out)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

[(10, 100, 21), (10, 100, 4)]

2-5. FCOS

In [12]:
#train model
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, frozen_stages = 1, weights = "imagenet_v2")

out = tfdet.model.detector.fcos(feature, image_shape = tf.shape(x)[1:3], n_class = n_class)

model = tfdet.model.train.fcos.train_model(x, *out,
                                           proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)
model.compile()
model.evaluate(bbox_pipe)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

1/1 [==============================] - 6s 6s/step - loss: 14739.4541 - loss_class: 14732.3281 - loss_bbox: 4.7866 - loss_conf: 2.3385


[(10, 100, 21), (10, 100, 4)]

In [13]:
#predict model
nms_out = tfdet.model.postprocess.fcos.FilterDetection(proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)(out)
model = tf.keras.Model(x, nms_out)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

[(10, 100, 21), (10, 100, 4)]

## 3. Build 2-Stage Object Detector

3-1. Faster R-CNN

In [14]:
#train model
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, frozen_stages = 1, weights = "imagenet_v2")

out = tfdet.model.detector.faster_rcnn(feature, image_shape = tf.shape(x)[1:3], n_class = n_class, scale = [32, 64, 128, 256, 512], ratio = [0.5, 1, 2],
                                       train = True)

model = tfdet.model.train.rcnn.train_model(x, *out,
                                           proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)
model.compile()
model.evaluate(bbox_pipe)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

1/1 [==============================] - 4s 4s/step - loss: 5518.9307 - loss_rpn_class: 3061.1157 - loss_rpn_bbox: 2.6287 - loss_roi_class: 2453.4419 - loss_roi_bbox: 1.7446


[(10, 100, 21), (10, 100, 4)]

In [15]:
#predict model
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, frozen_stages = 1, weights = "imagenet_v2")

out = tfdet.model.detector.faster_rcnn(feature, image_shape = tf.shape(x)[1:3], n_class = n_class, scale = [32, 64, 128, 256, 512], ratio = [0.5, 1, 2])
nms_out = tfdet.model.postprocess.rcnn.FilterDetection(proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)(out)
model = tf.keras.Model(x, nms_out)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

[(10, 100, 21), (10, 100, 4)]

3-2. Mask R-CNN

In [16]:
#train model
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, frozen_stages = 1, weights = "imagenet_v2")

out = tfdet.model.detector.mask_rcnn(feature, image_shape = tf.shape(x)[1:3], n_class = n_class, scale = [32, 64, 128, 256, 512], ratio = [0.5, 1, 2],
                                     train = True, mask_pool_size = 7)

model = tfdet.model.train.rcnn.train_model(x, *out,
                                           proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)
model.compile()
model.evaluate(mask_pipe)
[p.shape for p in model.predict(mask_pipe, verbose = 0)]

1/1 [==============================] - 10s 10s/step - loss: 7354.8213 - loss_rpn_class: 4646.4443 - loss_rpn_bbox: 2.3232 - loss_roi_class: 2704.1577 - loss_roi_bbox: 1.1872 - loss_roi_mask: 0.7090


[(10, 100, 21), (10, 100, 4), (10, 100, 14, 14, 1)]

In [17]:
#predict model
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, frozen_stages = 1, weights = "imagenet_v2")

out = tfdet.model.detector.mask_rcnn(feature, image_shape = tf.shape(x)[1:3], n_class = n_class, scale = [32, 64, 128, 256, 512], ratio = [0.5, 1, 2], 
                                     mask_pool_size = 7)
nms_out = tfdet.model.postprocess.rcnn.FilterDetection(proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)(out)
model = tf.keras.Model(x, nms_out)
[p.shape for p in model.predict(mask_pipe, verbose = 0)]

[(10, 100, 21), (10, 100, 4), (10, 100, 14, 14, 1)]

3-3. Cascade R-CNN

In [18]:
#train model
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, frozen_stages = 1, weights = "imagenet_v2")

out = tfdet.model.detector.cascade_rcnn(feature, image_shape = tf.shape(x)[1:3], n_class = n_class, scale = [32, 64, 128, 256, 512], ratio = [0.5, 1, 2],
                                        train = True)

model = tfdet.model.train.rcnn.train_model(x, *out,
                                           proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)
model.compile()
model.evaluate(bbox_pipe)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

1/1 [==============================] - 7s 7s/step - loss: 8873.5205 - loss_rpn_class: 3283.8320 - loss_rpn_bbox: 1.6162 - loss_roi_class_1: 3511.5962 - loss_roi_bbox_1: 0.0000e+00 - loss_roi_class_2: 1289.3551 - loss_roi_bbox_2: 0.0000e+00 - loss_roi_class_3: 787.1217 - loss_roi_bbox_3: 0.0000e+00


[(10, 100, 21), (10, 100, 4)]

In [19]:
#predict model
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, frozen_stages = 1, weights = "imagenet_v2")

out = tfdet.model.detector.cascade_rcnn(feature, image_shape = tf.shape(x)[1:3], n_class = n_class, scale = [32, 64, 128, 256, 512], ratio = [0.5, 1, 2])
nms_out = tfdet.model.postprocess.rcnn.FilterDetection(proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)(out)
model = tf.keras.Model(x, nms_out)
[p.shape for p in model.predict(bbox_pipe, verbose = 0)]

[(10, 100, 21), (10, 100, 4)]

3-4. Hybrid Task Cascade

In [20]:
#train model
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, frozen_stages = 1, weights = "imagenet_v2")

out = tfdet.model.detector.hybrid_task_cascade(feature, image_shape = tf.shape(x)[1:3], n_class = n_class, scale = [32, 64, 128, 256, 512], ratio = [0.5, 1, 2],
                                               train = True, mask_pool_size = 7)

model = tfdet.model.train.rcnn.train_model(x, *out,
                                           proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)
model.compile()
model.evaluate(mask_pipe)
[p.shape for p in model.predict(mask_pipe, verbose = 0)]

1/1 [==============================] - 29s 29s/step - loss: 5615.0259 - loss_rpn_class: 1407.0381 - loss_rpn_bbox: 2.8818 - loss_roi_class_1: 1600.0781 - loss_roi_bbox_1: 2.9274 - loss_roi_mask_1: 1.3057 - loss_roi_class_2: 2140.2781 - loss_roi_bbox_2: 0.8526 - loss_roi_mask_2: 0.1734 - loss_roi_class_3: 456.9531 - loss_roi_bbox_3: 0.0000e+00 - loss_roi_mask_3: 0.0000e+00 - loss_semantic: 2.5374


[(10, 100, 21), (10, 100, 4), (10, 100, 14, 14, 1)]

In [21]:
#predict model
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, frozen_stages = 1, weights = "imagenet_v2")

out = tfdet.model.detector.hybrid_task_cascade(feature, image_shape = tf.shape(x)[1:3], n_class = n_class, scale = [32, 64, 128, 256, 512], ratio = [0.5, 1, 2], 
                                               mask_pool_size = 7)
nms_out = tfdet.model.postprocess.rcnn.FilterDetection(proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.05)(out)
model = tf.keras.Model(x, nms_out)
[p.shape for p in model.predict(mask_pipe, verbose = 0)]

[(10, 100, 21), (10, 100, 4), (10, 100, 14, 14, 1)]